In [ ]:
import os
import pandas as pd
import numpy as np 

In [ ]:
# load dataset
path = '../data/cleanpenelopedict.json'
penelope = pd.read_json(path)
penelope['notes'] = penelope['notes'].str.strip()

In [ ]:
# load relevant tables
path = '../data/relevant_tables.csv'
relevant_tbl = pd.read_csv(path)
relevant_tbl['rpt_name'] = relevant_tbl['rpt_name'].str.strip()

In [ ]:
# select only relevant tables from penelope df
sjpc_pnlp = penelope.merge(relevant_tbl, on='rpt_name')
sjpc = penelope[penelope['rpt_name'].isin(sjpc_pnlp['rpt_name'])]

In [ ]:
# sanity check
len(sjpc.rpt_name.unique())

In [ ]:
# replace blanks with nan
sjpc['notes'] = sjpc.notes.replace('',np.nan, regex=True)
sjpc.notes.fillna(value=np.nan, inplace=True)
sjpc = sjpc.sort_values(by=['field_name','notes'])

In [ ]:
# fill blanks by group
sjpc['notes'] = sjpc.groupby('field_name')['notes'].ffill()

In [ ]:
# implode
sjpc_dict = (sjpc.groupby(['field_name', 'data_type']).agg({'rpt_name': lambda x: x.tolist(), 'category': lambda x: x.tolist(), 'notes': lambda x: x.tolist()})
             .rename({'rpt_name': 'rpt_tables'} , axis=1)
             .reset_index())


In [ ]:
# keep only unique values in the lists
sjpc_dict['notes'] = sjpc_dict['notes'].apply(np.unique)

In [ ]:
# convert the list columns to string
sjpc_dict['rpt_tables'] = [','.join(map(str, l)) for l in sjpc_dict['rpt_tables']]
sjpc_dict['category'] = [','.join(map(str, l)) for l in sjpc_dict['category']]
sjpc_dict['notes'] = [','.join(map(str, l)) for l in sjpc_dict['notes']]
sjpc_dict['notes'] = sjpc_dict['notes'].replace('nan','')
sjpc_dict['notes'] = sjpc_dict['notes'].replace('nan,nan','')

In [ ]:
# resulting data dictionary template for sjpc
sjpc_dict = sjpc_dict[['category','field_name','data_type', 'rpt_tables', 'notes']]
sjpc_dict['description'] = ''
sjpc_dict.head(30)

In [ ]:
# export as csv
path = '../data/sjpc_datadict.csv'
sjpc_dict.to_csv(path, index=False)